In [43]:
import scipy.io
import scipy.signal
import numpy as np
mat = scipy.io.loadmat('sintesis.mat')
mat.pop('__header__')
mat.pop('__version__')
mat.pop('__globals__')

B = mat['ganancia'][0]
a_i = mat['coef']
x_n = mat['excit']

In [48]:
y_n = scipy.signal.lfilter(B, a_i[99], x_n[99], axis= -1, zi=None)
y_n

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.85768166e-04,
        7.85173044e-04,  1.48416917e-03,  2.29478057e-03,  3.23951661e-03,
        4.24099220e-03,  5.28011921e-03,  6.35608807e-03,  7.38530346e-03,
        8.29640490e-03,  9.04433903e-03,  9.58421423e-03,  9.89338530e-03,
        9.95407707e-03,  9.79302588e-03,  9.42668757e-03,  8.92819890e-03,
        8.51478060e-03,  8.80891794e-03,  1.10871613e-02,  1.48749046e-02,
        1.96261693e-02,  